In [59]:
import pandas as pd
import matplotlib.pyplot as plt

In [60]:
min_val = 0
max_val = 100

opcash_to_capex = pd.read_csv('opcash_to_capex_tech.csv').drop(columns=['Unnamed: 0'])
opcash_to_capex = opcash_to_capex[(opcash_to_capex['Operating cash flow to capex'] <= max_val)&
                                  (opcash_to_capex['Operating cash flow to capex'] >= min_val) &
                                  ~(opcash_to_capex['capex'] == 0)]
opcash_to_capex.sort_values(by='Operating cash flow to capex', ascending=True)
annual_returns = pd.read_csv('annual_returns.csv').rename(columns={'Date':'date', 'Close': 'close'})

In [61]:
annual_returns['date'] = pd.to_datetime(annual_returns['date']).dt.year
annual_returns

,date,close,ticker
0,1994,0.149797,AAPL
1,1995,-0.190334,AAPL
2,1996,-0.538918,AAPL
3,1997,-0.599993,AAPL
4,1998,0.603054,AAPL
...,...,...,...
5971,2024,-0.158824,TYGO
5972,2024,0.000000,AOGOW
5973,2022,0.141991,WAVS
5974,2023,0.034070,WAVS


In [62]:
opcash_to_capex = opcash_to_capex[opcash_to_capex.groupby('ticker')['ticker'].transform('count') >= 10]
opcash_to_capex

,date,Operating cash flow,ticker,capex,Operating cash flow to capex
0,2023,1.105430e+11,AAPL,-1.095900e+10,9.913789
1,2022,1.221510e+11,AAPL,-1.070800e+10,8.766199
2,2021,1.040380e+11,AAPL,-1.108500e+10,10.654761
3,2020,8.067400e+10,AAPL,-7.309000e+09,9.059920
4,2019,6.939100e+10,AAPL,-1.049500e+10,15.124440
...,...,...,...,...,...
3844,2015,2.864500e+07,CAMP,-7.437000e+06,25.962646
3845,2014,2.281600e+07,CAMP,-2.133000e+06,9.348703
3846,2013,1.659700e+07,CAMP,-1.852000e+06,11.158643
3847,2012,1.243200e+07,CAMP,-1.076000e+06,8.655084


In [63]:
opcash_to_capex['rank'] = opcash_to_capex.groupby('date')['Operating cash flow to capex'].rank(ascending=False)
opcash_to_capex

,date,Operating cash flow,ticker,capex,Operating cash flow to capex,rank
0,2023,1.105430e+11,AAPL,-1.095900e+10,9.913789,34.0
1,2022,1.221510e+11,AAPL,-1.070800e+10,8.766199,81.0
2,2021,1.040380e+11,AAPL,-1.108500e+10,10.654761,76.0
3,2020,8.067400e+10,AAPL,-7.309000e+09,9.059920,81.0
4,2019,6.939100e+10,AAPL,-1.049500e+10,15.124440,66.0
...,...,...,...,...,...,...
3844,2015,2.864500e+07,CAMP,-7.437000e+06,25.962646,45.0
3845,2014,2.281600e+07,CAMP,-2.133000e+06,9.348703,91.0
3846,2013,1.659700e+07,CAMP,-1.852000e+06,11.158643,85.0
3847,2012,1.243200e+07,CAMP,-1.076000e+06,8.655084,93.0


In [64]:
opcash_to_capex['quintile'] = pd.qcut(opcash_to_capex['rank'], q=5, labels=False, duplicates='drop') + 1


In [65]:
quintile_portfolios = {}

for q in range(1, 6):
    quintile_portfolios[q] = opcash_to_capex[opcash_to_capex['quintile'] == q].copy()


In [66]:
# pd.reset_option('display.max_rows', None)
opcash_to_capex[#(opcash_to_capex['date']==2009) &
                (opcash_to_capex['quintile']==1)].sort_values(by='date')


,date,Operating cash flow,ticker,capex,Operating cash flow to capex,rank,quintile
3397,1998,6.283000e+06,RELL,-4.116000e+06,65.510107,1.0,1
2374,1998,2.066091e+07,PLUS,-3.097322e+06,14.991216,3.0,1
2707,1998,1.761000e+07,SPNS,-6.042000e+06,34.310051,2.0,1
3551,1999,-7.844000e+06,AXTI,-7.527000e+06,95.958695,1.0,1
3754,1999,4.225000e+06,KTCC,-2.288000e+06,54.153846,2.0,1
...,...,...,...,...,...,...,...
1342,2023,3.839500e+07,GWRE,-1.742700e+07,45.388722,10.0,1
3836,2023,-2.293200e+07,CAMP,-1.110000e+07,48.403977,9.0,1
2089,2023,2.351680e+08,SANM,-1.913670e+08,81.374592,3.0,1
919,2023,1.734000e+09,JBL,-1.030000e+09,59.400231,6.0,1


In [67]:
opcash_to_capex[(opcash_to_capex['date']==2009) &
                (opcash_to_capex['quintile']==1)].sort_values(by='rank')

,date,Operating cash flow,ticker,capex,Operating cash flow to capex,rank,quintile
1377,2009,261725000.0,AMKR,-218177000.0,83.361162,1.0,1
2728,2009,11582000.0,UCTT,-9448000.0,81.574858,2.0,1
1994,2009,37249000.0,TSEM,-28066000.0,75.346989,3.0,1
289,2009,332665000.0,AMAT,-248427000.0,74.677829,4.0,1
1894,2009,9190000.0,AEIS,-5611000.0,61.055495,5.0,1
2851,2009,12817000.0,ERII,-7687000.0,59.975033,6.0,1
340,2009,-78129000.0,LRCX,-44282000.0,56.678058,7.0,1
933,2009,557309000.0,JBL,-292238000.0,52.437337,8.0,1
3061,2009,7534000.0,MGIC,-3708000.0,49.216883,9.0,1
3541,2009,4665000.0,AXTI,-1963000.0,42.079314,10.0,1


In [58]:
returns = opcash_to_capex.merge(annual_returns, how='inner', on=['ticker', 'date'])
avg_returns = returns.groupby('quintile')['close'].mean()

returns_roll_3 = returns.groupby(['quintile', 'date'])['close'].rolling(window=3).mean().reset_index()
returns_roll_3
returns

,date,Operating cash flow,ticker,capex,Operating cash flow to capex,rank,quintile,close
0,2023,1.105430e+11,AAPL,-1.095900e+10,9.913789,34.0,2,0.353998
1,2022,1.221510e+11,AAPL,-1.070800e+10,8.766199,81.0,4,-0.392748
2,2021,1.040380e+11,AAPL,-1.108500e+10,10.654761,76.0,4,0.275682
3,2020,8.067400e+10,AAPL,-7.309000e+09,9.059920,81.0,4,0.438958
4,2019,6.939100e+10,AAPL,-1.049500e+10,15.124440,66.0,4,0.470178
...,...,...,...,...,...,...,...,...
1630,2015,2.864500e+07,CAMP,-7.437000e+06,25.962646,45.0,3,0.079277
1631,2014,2.281600e+07,CAMP,-2.133000e+06,9.348703,91.0,5,-0.480874
1632,2013,1.659700e+07,CAMP,-1.852000e+06,11.158643,85.0,4,0.691813
1633,2012,1.243200e+07,CAMP,-1.076000e+06,8.655084,93.0,5,0.486779


In [11]:
excess_returns = pd.DataFrame(avg_returns-returns['close'].mean()).rename(columns={'close':'excess returns'})
excess_returns.merge(avg_returns, on='quintile').to_csv('excess_returns.csv')

In [12]:
avg_returns = returns.groupby(['date', 'quintile'])['close'].mean()
pd.DataFrame(avg_returns)

close
date quintile          
1998 1        -0.115616
1999 1         0.323522
2000 1        -7.475483
2001 1        -0.613106
2002 1        -1.783651
...                 ...
2022 4        -0.631526
     5        -0.346227
2023 1        -0.748423
     2         0.208146
     3         0.143868

[84 rows x 1 columns]

In [13]:
returns['close'].mean()

-0.003730185853014247

## CAGR

In [28]:
def cagr(beginning_value, ending_value, years):
    cagr_val = (ending_value / beginning_value) ** (1 / years) - 1
    return cagr_val
yearly_avg_returns = pd.DataFrame(returns.groupby(['date', 'quintile'])['close'].mean()).reset_index()
yearly_avg_returns = yearly_avg_returns[(yearly_avg_returns['date']>=2009)]
yearly_avg_returns

,date,quintile,close
12,2009,1,0.389125
13,2009,2,0.431270
14,2009,3,0.384919
15,2009,4,0.457813
16,2010,1,0.160521
...,...,...,...
79,2022,4,-0.631526
80,2022,5,-0.346227
81,2023,1,-0.748423
82,2023,2,0.208146


In [54]:
years = 2023-2019
for i in range(1,6):
    print (i)
    start_val = yearly_avg_returns[yearly_avg_returns['quintile'] == i].reset_index().loc[0, 'close']
    end_val = yearly_avg_returns[yearly_avg_returns['quintile'] == i].reset_index().iloc[-1]['close']
    print(cagr(start_val, end_val, years))

1
nan
2
-0.16650206648662003
3
-0.21810373399267768
4
nan
5
nan


/var/folders/_p/y56kr6vs1fz7ycsbxhyqwwwc0000gn/T/ipykernel_65913/3499885254.py:2: RuntimeWarning: invalid value encountered in scalar power
  cagr_val = (ending_value / beginning_value) ** (1 / years) - 1
/var/folders/_p/y56kr6vs1fz7ycsbxhyqwwwc0000gn/T/ipykernel_65913/3499885254.py:2: RuntimeWarning: invalid value encountered in scalar power
  cagr_val = (ending_value / beginning_value) ** (1 / years) - 1
/var/folders/_p/y56kr6vs1fz7ycsbxhyqwwwc0000gn/T/ipykernel_65913/3499885254.py:2: RuntimeWarning: invalid value encountered in scalar power
  cagr_val = (ending_value / beginning_value) ** (1 / years) - 1


In [56]:
start_val = yearly_avg_returns[yearly_avg_returns['quintile'] == 1].reset_index().loc[0, 'close']
end_val = yearly_avg_returns[yearly_avg_returns['quintile'] == 1].reset_index().iloc[-1]['close']
print(cagr(start_val, end_val, years))
print(start_val, end_val, years)

nan
0.3891249488755727 -0.7484232400287456 4


/var/folders/_p/y56kr6vs1fz7ycsbxhyqwwwc0000gn/T/ipykernel_65913/3499885254.py:2: RuntimeWarning: invalid value encountered in scalar power
  cagr_val = (ending_value / beginning_value) ** (1 / years) - 1


In [57]:
yearly_avg_returns[yearly_avg_returns['quintile'] == 1]

,date,quintile,close
12,2009,1,0.389125
16,2010,1,0.160521
21,2011,1,-1.050471
26,2012,1,0.086162
31,2013,1,0.292581
36,2014,1,0.052632
41,2015,1,-0.315685
46,2016,1,0.160418
51,2017,1,-0.023562
56,2018,1,-0.436766
